# Proyecto Information Retrieval

In [71]:
import textract
import os
import tensorflow_text
from os import listdir
from os.path import isfile, join
import nltk
from nltk.tokenize import sent_tokenize
import string
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import word_tokenize  
import os
import pandas as pd
import re
import seaborn as sns
from nltk.corpus import stopwords
from collections import OrderedDict 
import re

stopWords = stopwords.words('spanish')
not_stopwords = {'no'} 
stopWords = set([word for word in stopWords if word not in not_stopwords])
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
path_epub = 'LIBROS'
descriptores_dir = 'descriptores'
texto_dir = 'texto'
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3" #CNN ARCHITECTURE
#module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3" #TRANSFORMER ARCHITECTURE
model = hub.load(module_url)

In [72]:
sinStopWords = True # eliminar stopwords

In [73]:
#nltk.download('stopwords')
#nltk.download('punkt')

In [74]:
if not os.path.exists(descriptores_dir):
    os.makedirs(descriptores_dir)
    
if not os.path.exists(texto_dir):
    os.makedirs(texto_dir)

In [105]:
def embed(input):
    '''
    Funcion recibe una lista de strings y retorna una embedding de 1 x 512
    para cada una de las frases
    -----------------------------------
    input: lista de strings, n elementos.
    output: tensor de dimensiones n x 512    
    
    '''
    return model(input)

def obtenerFrases(texto, eliminarStopWords, separador='.'):
    '''
    Funcion que recibe una string de tamaño variable y lo separa
    en frases dividiendo el texto por separador
    -----------------------------------
    Input:
        texto: String 
        eliminarStopWords: Booleando que indica si se desea eliminar stopwords
    Output:
        frases_clean: Lista de strings limpios, que corresponden a todas las frases
    '''
    
    frases = texto.split(separador)
    frases_clean =[]

    lista_simbolos = list(string.punctuation) + ['¡','¿','-','—','«','»','–','”','“']
    for frase in frases:
        frase = frase.replace('\n', ' ')
        frase_clean = "".join([i.lower() for i in frase if ((i not in lista_simbolos) and (not i.isdigit()))])
        frase_clean = re.sub(' +', ' ', frase_clean)
        if frase_clean=='' or len(frase_clean.split())<5:
            continue            
        if eliminarStopWords:
            word_tokens = word_tokenize(frase_clean)    
            frase_clean = " ".join([w for w in word_tokens if not w in stopWords])  
        frases_clean.append(frase_clean.strip())
      
        
    return frases_clean
    
def obtenerEmbeddings(frases):
    '''
    Calcula la embedding para una lista de frases.
    -----------------------------------
    Input:
        frases: Lista de n strings
    Output:
        message_embeddings: tensor de dimensiones n x 512      
    '''
    
    message_embeddings = embed(frases)

    return message_embeddings


In [106]:
class Libro:
    def __init__(self,titulo, eliminarStopWords,separador='.'):
        self.texto = textract.process(os.path.join(path_epub,titulo),encoding='utf_8').decode('UTF-8')
        self.frases_clean  = obtenerFrases(self.texto, eliminarStopWords,separador)
        self.frases  = obtenerFrases(self.texto, False,separador)
        self.embeddings = obtenerEmbeddings(self.frases_clean)
        
    

# EXTRAER TEXTO, LIMPIAR Y OBTENER EMBEDDINGS

In [107]:

lista_libros_procesados = []
def procesar_libros(path_epub,separador):
    '''
    Funcion que extrae el texto de cada epub. Se extraen las frases y 
    se guardan en un archivo de texto. A cada frase se le calcula su embedding y se guarda una matrix con 
    todas las frases.
    '''
    lista_libros_epub= [f for f in listdir(path_epub) if isfile(join(path_epub, f))]

    n = len(lista_libros_epub)
    print(f'Se han encontrado {n} libros')
    i = 0
    for file in lista_libros_epub:
        try:
            text_file = open(texto_dir+'/' + file+ ".txt", "w",encoding='utf-8')
            libro = Libro(file, eliminarStopWords =sinStopWords, separador=separador)
            for frase in libro.frases:
                text_file.write(frase + '\n')
            text_file.close()
            np.save(descriptores_dir +'/' + file, libro.embeddings)
            print(file)
            lista_libros_procesados.append(file)
            i=i+1
        except:
            print('Fallo:', file)
        

    print(f'Se han procesado {i}/{n} epubs con exito')

In [108]:
procesar_libros(path_epub,separador='.')

Se han encontrado 31 libros
El amor en los tiempos del colera - Gabriel Garcia Marquez.epub
El Club de la Lucha - Chuck Palahniuk.epub
El Codigo Da Vinci - Dan Brown (13).epub
El diario de Ana Frank - Anne Frank.epub
El Gran Gatsby - Francis Scott Fitzgerald.epub
El hobbit - J. R. R. Tolkien.epub
El principito - Antoine de Saint-Exupery.epub
El_retorno_del_rey_J_R_R_Tolkien.epub
Fundación y Tierra - Isaac Asimov.epub
Hacia la Fundación - Isaac Asimov.epub
Harry Potter y el caliz de fueg - J.K. Rowling.epub
Harry Potter y el legado maldito - J.R. Rowling, Jhon Tiffany y Jack Thorne.epub
Harry Potter y el misterio del - J.K. Rowling.epub
Harry Potter y el prisionero de - J.K. Rowling.epub
Harry Potter y la camara secret - J.K. Rowling.epub
Harry Potter y la piedra filoso - J.K. Rowling.epub
Harry Potter y las Reliquias de - J.K. Rowling.epub
Historia universal Asimov La Republica Romana.epub
Historia universal Asimov_ La Republica.epub
historia_de_dos_ciudades.epub
Hombre y superhombre -

In [109]:
libro = Libro('viaje_al_centro_de_la_tierra.epub', eliminarStopWords =sinStopWords)

# Utilidad para buscar la frase dentro de todos los libros. Elegir similitud coseno mas alta

In [110]:
def buscar(cita,lista_libros_procesados):
    '''
    La cita es buscada en cada libro que ha sido procesado. Se calcula para cada libro
    la frase mas semejante con respecto a la cita de acuerdo con la similitud coseno
    '''
    frases_libros = lista_libros_procesados
    parecidas = {}
    for titulo in frases_libros:
        query_vec = model(cita) #encodes the string in the list
        file = open(os.path.join(texto_dir , titulo +'.txt'), encoding='utf-8')
        all_lines = file.readlines()        
        bank_vec = np.load(os.path.join(descriptores_dir , titulo +'.npy'))
        correlation = np.transpose(np.inner(query_vec,bank_vec))
        idx_max = np.argmax(correlation, axis=0)[0]
        #print(idx_max)
        #print(len(all_lines))
        parecida = all_lines[idx_max].replace('\n','')
        corr =float("%.4f" % max(correlation)[0])        
        parecidas[corr] = [titulo, parecida]
    parecidas = OrderedDict(sorted(parecidas.items(), reverse = True))
    primero = list(parecidas.keys())[0]
    return primero, parecidas[primero]
        

# INFERIR LIBROS A PARTIR DE UN ARCHIVO CON MULTIPLES CITAS

# 

In [111]:
def buscar_citas_from_file(file_name):
    file = open(file_name, encoding='utf-8')
    all_lines = file.readlines()   

    text_file = open( "resultados.txt", "w",encoding='utf-8')
    for line in all_lines:    
        query = line.replace('.','').strip()
        print(f'Buscando: "{query}"')
        buscar_frase = obtenerFrases(query, eliminarStopWords=sinStopWords)
        correlacion, [titulo, cita ] = buscar(buscar_frase,lista_libros_procesados)
        text_file.write(query + '\t' + cita+ '\t' + titulo  +  '\t' + str(correlacion)+ '\n')
    text_file.close()
    
buscar_citas_from_file('citas.txt')

Buscando: "cuando te hayas consolado te alegrarás de haberme conocido"
Buscando: "Quien no es capaz de desprenderse de un tesoro en un momento de necesidad es como un esclavo encadenado"
Buscando: "Mientras el corazón late, mientras el cuerpo y alma siguen juntos, no puedo admitir que cualquier criatura dotada de voluntad tiene necesidad de perder la esperanza en la vida"
Buscando: "¡Qué maravilloso es que nadie necesite esperar ni un solo momento antes de comenzar a mejorar el mundo!"
Buscando: "No existe el bien, ni el mal, solo hay poder y personas demasiado débiles para buscarlo"
Buscando: "Nunca es demasiado tarde para ser sabio"
Buscando: "El hombre llega mucho más lejos para evitar lo que teme que para alcanzar lo que desea"
Buscando: "No todos los que vagan están perdidos"
Buscando: "Solo después de haber perdido todo, somos libres de hacer cualquier cosa"
Buscando: "El hombre razonable se adapta al mundo: el hombre no razonable persiste en intentar adaptar el mundo a sí mismo 

# EVALUAR INFERENCIA CON RESPECTO A LAS ETIQUETAS VERDADERAS

In [115]:

def evaluar_inferencias(resultados,ground_true):
    file_resultados = open(resultados, encoding='utf-8')
    lines_resultados = file_resultados.readlines()    
    file_ground_true = open(ground_true, encoding='utf-8')
    lines_ground_true = file_ground_true.readlines()   

    correctas = 0
    for line_idx in range(len(lines_resultados)):
        cita = lines_resultados[line_idx].split('\t')[0].strip()
        encontrada =  lines_resultados[line_idx].split('\t')[1].strip()
        predicted= lines_resultados[line_idx].split('\t')[2].strip()
        corr = lines_resultados[line_idx].split('\t')[3].strip()
        ground_true = lines_ground_true[line_idx].split('\t')[1].strip()
        if predicted == ground_true:
            print('CORRECTO')

            correctas +=1
        else:
            print('INCORRECTO')
        print('Libro predicho: ',predicted)
        print('Libro verdadero: ',ground_true)
        print('Cita buscada: "',cita,'"')
        print('Cita encontrada: "',encontrada,'"')
        print('Similitud Coseno:',corr)
        print('--------------------')
    precision = correctas/len(lines_resultados)
    print(f'Precision: {precision}')
    


In [116]:
evaluar_inferencias('resultados.txt','citas_ground_true.txt' )

CORRECTO
Libro predicho:  El principito - Antoine de Saint-Exupery.epub
Libro verdadero:  El principito - Antoine de Saint-Exupery.epub
Cita buscada: " cuando te hayas consolado te alegrarás de haberme conocido "
Cita encontrada: " cuando te hayas consolado siempre se consuela uno estarás contento de haberme conocido "
Similitud Coseno: 0.7821
--------------------
CORRECTO
Libro predicho:  Las_dos_torres_J_R_R_Tolkien.epub
Libro verdadero:  Las_dos_torres_J_R_R_Tolkien.epub
Cita buscada: " Quien no es capaz de desprenderse de un tesoro en un momento de necesidad es como un esclavo encadenado "
Cita encontrada: " quien no es capaz de desprenderse de un tesoro en un momento de necesidad es como un esclavo encadenado "
Similitud Coseno: 1.0
--------------------
INCORRECTO
Libro predicho:  El_retorno_del_rey_J_R_R_Tolkien.epub
Libro verdadero:  viaje_al_centro_de_la_tierra.epub
Cita buscada: " Mientras el corazón late, mientras el cuerpo y alma siguen juntos, no puedo admitir que cualquier

# Test para una cita

In [134]:
query = 'El que es feliz hace feliz a los demás nunca estará  desgracia. '
#query = 'La hoja es mucho más paciente'
#query = 'el avance de la civilización no es más que un ejercicio en la limitación de la intimidad'

buscar_frase = obtenerFrases(query,eliminarStopWords= sinStopWords)
corr, [titulo, cita ] = buscar(buscar_frase,lista_libros_procesados)
vector  = obtenerEmbeddings(buscar_frase)
print(titulo)
#print('Buscando: ','"',buscar_frase[0],'"')
print('Encontrado:','"',cita,'"')
print('Similitud Coseno:',corr)
#print(vector)

El diario de Ana Frank - Anne Frank.epub
Encontrado: " y el que es feliz hace feliz a los demás el que tiene valor y fe nunca estará sumido en la desgracia "
Similitud Coseno: 0.8485
